# OMOP to Hospital Patient Records

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

In [ ]:
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Visit Ocurrence to Hospital Patient Records

In [ ]:
query="""
    SELECT 
        v.patient,
        v.hospital,
        MIN(v.date) AS earliest_date
    FROM 
        visit_occurrence 
    WHERE visit_concept_id=38004515
    GROUP BY 
        v.patient, 
        v.hospital;
"""

insert_query = """
    INSERT INTO hospital_data (id, date_of_first_contact_with_the_hospital, hospital_data, patient)
    VALUES (%s, %s)
"""

curOMOP.execute(query)

results = curOMOP.fetchall()

for row in results:
    newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
    date_of_first_contact_with_the_hospital=row[2]
    hospital_data=row[1]
    patient = row[0]
    curIDEA.execute(insert_query, (newId, date_of_first_contact_with_the_hospital, hospital_data, patient))
    conn2.commit()
